## Import Packages

In [1]:
import os
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import bs4
import numpy as np

import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium import webdriver

## Prepare

In [4]:
STORAGE_DIR = "../input"

In [6]:
OUTPUT_DIR = "../output"

In [8]:
DOC_DIR = "../docu"

### news paper url dictionaries

In [11]:
# List of news pages to be scraped
newspaper_urls = dict(
    sz="https://www.sueddeutsche.de/thema/Nahost",
    zeit="https://www.zeit.de/thema/israel",
    spiegel="https://www.spiegel.de/thema/nahost/p2/",
)

### target objects

In [14]:
content_dict = {}
text_dict = {}
log_list = []
failing_list = []

### date format

In [17]:
# Current date as string
now = datetime.now()
now_str = now.strftime("%Y-%m-%d")
print(now_str)

2025-11-14


### Data Extraction

#### HTML Roh-Datei exportieren

In [21]:
def scrape_website(name, url):
    # (1) Run request
    response = requests.get(url, allow_redirects=True)
    content = response.content
    text = response.text

    # (2) File name to store the raw HTML
    file_name = os.path.join(
        STORAGE_DIR,
        f"{now_str}-{name}.html",
    )

    # (3) Write raw HTML
    with open(file_name, "wb") as f:
        f.write(response.content)

    # (4) Fill content_dict and text_dict
    content_dict[name] = response.content
    text_dict[name] = response.text

    # (5) Fill log_list
    log_info = dict(
        name=name,
        date=now_str,
        file_name=file_name,
        status=response.status_code,
        url=response.url,
        encoding=response.encoding,
    )
    log_list.append(log_info)

**Ausführen**

In [23]:
for name, url in newspaper_urls.items():
    try:
        scrape_website(name, url)
    except:
        failing_list.append((name, url))

### HTML-Datei laden

#### sueddeutsche

In [25]:
sz="https://www.sueddeutsche.de/thema/Nahost"

In [116]:
sz_html = os.path.join("../input/2025-11-14-sz.html")

In [118]:
with open(sz_html, "r", encoding= "utf-8") as f:
    sz_text = f.read()

**Titel des Artikels abrufen**

In [120]:
sz_soup = BeautifulSoup(sz_text, "html.parser")

In [122]:
sz_article_tag = sz_soup.select('article.sz-teaserlist-element.css-epgeur')

In [ ]:
article_id = []
title = []
content_preview = []
datum = []

for article in sz_article_tag:
    
    id_tag = article.attrs.get('id', 'KEINE ID GEFUNDEN')
    article_id.append(id_tag)

    #Titel des Artikels
#------------------------------------------------------------------------------------------------------    
    title_tag = article.select('span.css-ornnzr')

    if title_tag:

        for t in title_tag:
        
            title.append(t.get_text(strip=True))
                    
    else: 
        title.append('NA')

    # Vorschau des Artikel Inhaltes
#------------------------------------------------------------------------------------------------------    
    preview_tag = article.select('p.css-a6ecqs')

    if preview_tag:
        for p in preview_tag:
        
            content_preview.append(p.get_text(strip=True))

    else:
        content_preview.append('NA')

    # Datum der Veröffentlichung
#------------------------------------------------------------------------------------------------------    
    datum_tag = article.select('span.css-0')

    
    if datum_tag:

        for d in datum_tag:
            
            datum.append(d.get_text(strip=True))

    else:
        datum.append('NA')

print(f"Gefundene Artikel: {len(article_id)}") # Geändert
print(title)
print(content_preview)
print(datum)

## Mit Selenium Navigation

In [166]:
# Geändert: 'ValueError' aus diesem Import entfernt
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException # Geändert

# Setup Selenium
now_str = datetime.now().strftime("%Y-%m-%d")
name = "sz"

driver = webdriver.Chrome()
driver.get(sz)

# Cookie-Banner-Selektor
try:
    wait = WebDriverWait(driver, 10)
    # Versuch, den Button über seine eindeutige ID zu finden
    cookie_button = wait.until(
        EC.element_to_be_clickable((By.ID, "sdd-cmp-modal-button-agree")) 
    )
    cookie_button.click()
    print("Cookie-Banner geklickt.")
    time.sleep(1) 
except TimeoutException:
    print("Kein Cookie-Banner gefunden (oder er war nicht klickbar). Fahre fort.")

#--------------------Laden der Seite--------------------+
target_date = datetime.strptime("08.10.2025", "%d.%m.%Y")
print(f"Suche gestartet. Stoppe, wenn Datum älter als {target_date.strftime('%d.%m.%Y')} ist.")

# Geändert: Wir definieren den Datums-Selektor für die Warte-Logik
date_selector = (By.CSS_SELECTOR, 'span.css-0')

while True:
    try:
        # Geändert: Zähle, wie viele Datums-Elemente BEREITS da sind
        count_before = len(driver.find_elements(*date_selector))
    
        wait = WebDriverWait(driver, 10)
        load_element = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "css-1kk8krp"))
        )

        driver.execute_script("arguments[0].scrollIntoView(true);", load_element)
        time.sleep(0.5) 

        driver.execute_script("arguments[0].click();", load_element)

        # Geändert: Warte, bis die ANZAHL DER DATEN zugenommen hat
        try:
            wait.until(
                lambda d: len(d.find_elements(*date_selector)) > count_before
            )
            print(f"Datumsanzahl erhöht auf: {len(driver.find_elements(*date_selector))}")
        except TimeoutException:
            print("Wartezeit überschritten, aber es wurden keine neuen Datums-Elemente geladen.")
            break 
            
        # Geändert: Datumsprüfung (jetzt sicher, da Daten geladen sind)
        all_dates = driver.find_elements(*date_selector) # Geändert: Verwende den Selektor
        if not all_dates:
            print("Keine Datumsangaben mehr gefunden.")
            break

        last_date_text = all_dates[-1].text.strip()

        try:
            current_date = datetime.strptime(last_date_text, "%d.%m.%Y")
            if current_date < target_date:
                print(f"STOPP: {last_date_text} ist älter als das Ziel.")
                break
        except ValueError: 
            pass 

    except TimeoutException:
        print("Button nicht mehr gefunden. Ende erreicht.")
        break


##--------------------Verlängerte Seite speichern--------------------

rohes_html = driver.page_source

driver.quit()

file_name = os.path.join(
    STORAGE_DIR,
    f"{now_str}-{name}.html",
)

try:
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(rohes_html)
    print(f"Erfolgreich gespeichert in: {file_name}")

except Exception as e:
    print(f"Fehler beim Speichern der Datei: {e}")


# Die Variable 'rohes_html' wird direkt verwendet,
# Datei zu laden.
sz_soup = BeautifulSoup(rohes_html, "html.parser")

sz_article_tag = sz_soup.select('article.sz-teaserlist-element.css-epgeur')

#-----------Seite Parsen------------

article_id = []
title = []
content_preview = []
datum = []

for article in sz_article_tag:
    
    id_tag = article.attrs.get('id', 'KEINE ID GEFUNDEN')
    article_id.append(id_tag)

    #Titel des Artikels
#------------------------------------------------------------------------------------------------------    
    title_tag = article.select('span.css-ornnzr')

    if title_tag:

        for t in title_tag:
        
            title.append(t.get_text(strip=True))
                    
    else: 
        title.append('NA')

    # Vorschau des Artikel Inhaltes
#------------------------------------------------------------------------------------------------------    
    preview_tag = article.select('p.css-a6ecqs')

    if preview_tag:
        for p in preview_tag:
        
            content_preview.append(p.get_text(strip=True))

    else:
        content_preview.append('NA')

    # Datum der Veröffentlichung
#------------------------------------------------------------------------------------------------------    
    datum_tag = article.select('span.css-0')

    
    if datum_tag:

        for d in datum_tag:
            
            datum.append(d.get_text(strip=True))

    else:
        datum.append('NA')

print(f"Gefundene Artikel: {len(article_id)}") # Geändert
print(title)
print(content_preview)
print(datum)

Kein Cookie-Banner gefunden (oder er war nicht klickbar). Fahre fort.
Suche gestartet. Stoppe, wenn Datum älter als 08.10.2025 ist.
Datumsanzahl erhöht auf: 108
Datumsanzahl erhöht auf: 158
STOPP: 30.9.2025 ist älter als das Ziel.
Erfolgreich gespeichert in: ../input/2025-11-14-sz.html
Gefundene Artikel: 50
['Zwischenfälle in Gaza', 'EU-Kommission billigt Übernahme von Covestro', 'Wie Netanjahu die Demokratie angreift', 'Pro-palästinensische Aktivisten auf Brandenburger Tor', 'NA', 'Ministerpräsident gewinnt Parlamentswahl', 'Klagen gegen Waffenlieferungen an Israel abgewiesen', 'IAEA: Iran verletzt Regeln', 'Lange Haftstrafen gegen Syrer wegen Kriegsverbrechen', 'NA', '„Dann müssen wir Teheran evakuieren“', 'NA', 'NA', 'Hamas übergibt Leiche', 'Was ist ein Konsulat?', '„Es kommen jetzt mehr Kinder mit Verbrennungen in die Klinik“', 'Iran will neues Konsulat in Bonn eröffnen', '„Ohne Staatsbürgerschaft hätte ich jetzt richtig Angst“', 'UN heben Sanktionen gegen al-Scharaa auf', 'NA', '

In [160]:
article_dict = {
    'id': article_id,
    'title': title,
    'preview': content_preview,
    'date': datum
}

In [162]:
sz_df = pd.DataFrame(article_dict)
sz_df

,id,title,preview,date
0,teaser-httpswwwsueddeutschedewirtschaftcovestr...,Araber dürfen früheren Dax-Konzern kaufen,Der Kunststoffhersteller Covestro aus Leverkus...,NA
1,teaser-httpswwwsueddeutschedepolitiknahost-zwi...,Zwischenfälle in Gaza,Trotz Waffenruhe ist palästinensischen Angaben...,Heute 17:21 Uhr
2,teaser-httpswwwsueddeutschedewirtschafteu-komm...,EU-Kommission billigt Übernahme von Covestro,Der Ölkonzern Adnoc aus Abu Dhabi hat sein Zie...,Heute 16:03 Uhr
3,teaser-httpswwwsueddeutschedepolitikisrael-dem...,Wie Netanjahu die Demokratie angreift,"Israels Ministerpräsident versucht, mit zahlre...",13.11.2025
4,teaser-httpswwwsueddeutschedepolitikberlin-pro...,Pro-palästinensische Aktivisten auf Brandenbur...,Drei pro-palästinensische Aktivisten haben am ...,13.11.2025
5,teaser-httpswwwsueddeutschedemeinungfluechtlin...,NA,Die Regierenden haben zwei Optionen: Sie könne...,13.11.2025
6,teaser-httpswwwsueddeutschedepolitikirak-minis...,Ministerpräsident gewinnt Parlamentswahl,Aus der Parlamentswahl im Irak ist die Koaliti...,12.11.2025
7,teaser-httpswwwsueddeutschedepolitikruestungse...,Klagen gegen Waffenlieferungen an Israel abgew...,Palästinenser im Gazastreifen sind mit ihrem V...,12.11.2025
8,teaser-httpswwwsueddeutschedepolitikatom-iaea-...,IAEA: Iran verletzt Regeln,Iran kommt seinen Informationspflichten zu ums...,12.11.2025
9,teaser-httpswwwsueddeutschedepolitikprozess-in...,Lange Haftstrafen gegen Syrer wegen Kriegsverb...,Das Oberlandesgericht München hat zwei Syrer w...,11.11.2025


#### Zeit

In [887]:
zeit = "https://www.zeit.de/thema/israel"

In [837]:
zeit_html = os.path.join("../input/2025-11-07-zeit.html")

In [842]:
with open(zeit_html, "r", encoding= "utf-8") as f:
    zeit_text = f.read()

In [1067]:
driver = webdriver.Chrome()

**Titel**

In [917]:
# Extra Large

driver.get(zeit)

time.sleep(3) 


html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

zeit_title_tag = soup.select('span.zon-teaser__title.zon-teaser__title--extralarge')
zeit_title_tag2 = soup.select('span.zon-teaser__title.zon-teaser__title--large')
zeit_title_tag3 = soup.select('span.zon-teaser__title')

print(zeit_title_tag)
print(zeit_title_tag2)
print(zeit_title_tag3)

[<span class="zon-teaser__title zon-teaser__title--extralarge">Festnahmen nach Störung beim Konzert des Israel Philharmonic Orchestra</span>]
[<span class="zon-teaser__title zon-teaser__title--large">Israel greift erneut Ziele im Südlibanon an</span>, <span class="zon-teaser__title zon-teaser__title--large">Als der Traum vom Frieden starb</span>, <span class="zon-teaser__title zon-teaser__title--large">Iran will Atomanlagen wieder aufbauen</span>, <span class="zon-teaser__title zon-teaser__title--large">Keiner wartet auf Deutschland</span>, <span class="zon-teaser__title zon-teaser__title--large">Bundesbildungsministerin lehnt ESC-Ausschluss Israels ab</span>, <span class="zon-teaser__title zon-teaser__title--large">USA sehen Waffenruhe durch israelische Angriffe nicht in Gefahr</span>, <span class="zon-teaser__title zon-teaser__title--large">Spätsommer in Tel Aviv, Winter im Wedding</span>]
[<span class="zon-teaser__title zon-teaser__title--extralarge">Festnahmen nach Störung beim Kon

In [907]:
title_attr_df(zeit_title_tag)

,titel
0,Festnahmen nach Störung beim Konzert des Israe...


**Content-Preview**

In [930]:
# Large

driver.get(zeit)

time.sleep(3) 


s_zeit_html = driver.page_source
zeit_soup = BeautifulSoup(html, 'lxml')

zeit_content_preview_tag = soup.select('p.zon-teaser__summary')

print(zeit_content_preview_tag)

[<p class="zon-teaser__summary">In der Pariser Philharmonie haben Zuschauer wiederholt das Konzert eines israelischen Sinfonieorchesters gestört. Die Polizei nahm vier Menschen in Gewahrsam.</p>, <p class="zon-teaser__summary">Laut US-Präsident Trump sollen schon "sehr bald" 20.000 internationale Soldaten nach Gaza entsandt werden. Ihre Hauptaufgabe soll die Entmilitarisierung des Gebiets sein.</p>, <p class="zon-teaser__summary">Alle lebenden Geiseln sind zurück, mit der Hamas herrscht Waffenstillstand. Israel könnte eigentlich aufatmen. Doch das Land steht vor einer historischen Zerreißprobe. </p>, <p class="zon-teaser__summary">Die israelische Armee hat nach eigenen Angaben im Libanon neue Angriffe gegen die Hisbollah gestartet. Zuvor rief sie Bewohner auf, Gebäude zu verlassen. Das Liveblog</p>, <p class="zon-teaser__summary">Die Linksjugend wirft Israel ein Apartheidsystem und Genozid in Gaza vor, die Parteispitze sieht darin übermäßige Kritik. Die Beschlusslage der Partei sei kla

In [931]:
title_attr_df(zeit_content_preview_tag)

,titel
0,In der Pariser Philharmonie haben Zuschauer wi...
1,"Laut US-Präsident Trump sollen schon ""sehr bal..."
2,"Alle lebenden Geiseln sind zurück, mit der Ham..."
3,Die israelische Armee hat nach eigenen Angaben...
4,Die Linksjugend wirft Israel ein Apartheidsyst...
5,Soldaten foltern einen inhaftierten Palästinen...
6,Gabriel Wolff arbeitet als Kalligraf und Tatto...
7,Vor dreißig Jahren tötete ein jüdischer Fundam...
8,Ein Video soll schwere Misshandlungen durch is...
9,Nach eigenen Angaben hat die Hamas die toten K...


**Datum**

In [1059]:
spiegel_date_tag = spiegel_soup.find_all('time', attrs={'datetime': True})

In [1069]:
# Large

driver.get(zeit)

time.sleep(3) 


s_zeit_html = driver.page_source
zeit_soup = BeautifulSoup(html, 'lxml')

zeit_date_tag = soup.select('time.zon-teaser__datetime')

print(zeit_date_tag)

[<time class="zon-teaser__datetime" datetime="2025-11-07T10:56:39+01:00">Vor 4 Stunden</time>, <time class="zon-teaser__datetime" datetime="2025-11-07T08:09:03+01:00">Vor 7 Stunden</time>, <time class="zon-teaser__datetime" datetime="2025-11-06T16:01:41+01:00">Vor 23 Stunden</time>, <time class="zon-teaser__datetime" datetime="2025-11-07T12:27:39+00:00">Vor 2 Stunden</time>, <time class="zon-teaser__datetime" datetime="2025-11-06T15:12:33+01:00">Vor 1 Tag</time>, <time class="zon-teaser__datetime" datetime="2025-11-04T16:57:05+01:00">Vor 2 Tagen</time>, <time class="zon-teaser__datetime" datetime="2025-11-04T14:26:01+01:00">4. November 2025</time>, <time class="zon-teaser__datetime" datetime="2025-11-04T06:58:03+01:00">4. November 2025</time>, <time class="zon-teaser__datetime" datetime="2025-11-03T10:00:55+01:00">3. November 2025</time>, <time class="zon-teaser__datetime" datetime="2025-11-02T21:31:48+01:00">2. November 2025</time>, <time class="zon-teaser__datetime" datetime="2025-11

In [1073]:
title_attr_df(zeit_date_tag)

,titel
0,Vor 4 Stunden
1,Vor 7 Stunden
2,Vor 23 Stunden
3,Vor 2 Stunden
4,Vor 1 Tag
5,Vor 2 Tagen
6,4. November 2025
7,4. November 2025
8,3. November 2025
9,2. November 2025


**Titel**

In [924]:
faz="https://www.faz.net/aktuell/politik/thema/nahostkonflikt"

In [926]:
faz_html = os.path.join("../input/2025-11-07-faz.html")

In [928]:
with open(faz_html, "r", encoding= "utf-8") as f:
    faz_text = f.read()

In [946]:
faz_title_tag = soup.find_all('h3')

In [948]:
title_attr_df(faz_title_tag)

,titel
0,"Nicht mehr suchen, Antworten finden"
1,\nPariser Philharmonie: \nFestnahmen nach Stör...
2,\nNaher Osten: \nUSA wollen Gaza-Friedensplan ...
3,\nPolitische Spaltung in Israel: \nIm Inneren ...
4,\n\nLiveblog:Krieg in Gaza: \nIsrael greift er...
5,\nDie Linke: \nParteivorstand der Linken kriti...
6,"\nVideoleak aus Sde Teiman: \n""Wir verlieren u..."
7,"\nGabriel Wolff: \n""Menschen, die sich tätowie..."
8,Podcast abonnieren
9,\nAttentat auf Izchak Rabin: \nAls der Traum v...


In [960]:
driver.get(faz)

time.sleep(3) 


s_faz_html = driver.page_source
faz_soup = BeautifulSoup(html, 'lxml')

faz_title_tag = soup.select('h3.text-22.font-serif')

print(faz_title_tag)

[]


In [ ]:
driver.get(faz)

time.sleep(3) 


s_faz_html = driver.page_source
faz_soup = BeautifulSoup(html, 'lxml')

faz_title_tag = soup.select('h3.item')

print(faz_title_tag)

**Content-Preview**

In [ ]:
az_soup.select('h3.text-22.font-serif')

In [966]:
ts="https://www.tagesspiegel.de/internationales/themen/krieg-in-nahost"

In [968]:
ts_html = os.path.join("../input/2025-11-07-ts.html")

In [970]:
with open(ts_html, "r", encoding= "utf-8") as f:
    ts_text = f.read()

In [983]:
ts_soup = BeautifulSoup(ts_text, "html.parser")

In [989]:
ts_title_tag = ts_soup.find_all('span', attrs={'data-kilkaya': True})

**Titel**

In [993]:
title_attr_df(ts_title_tag)

,titel
0,Nach Äußerungen zum Nahostkonflikt
1,Gruppe prügelt in Neukölln auf junge Männer ein
2,"Der Gaza-Krieg, Trump und der Nahe Osten"
3,"Ist es naiv, auf friedliche Zeiten zu hoffen?"
4,Luigi Toscano porträtiert die Opfer der Hamas
...,...
75,Unbekannte dringen ins Rathaus Spandau ein und...
76,Zwischenbilanz des Venedig Filmfestivals
77,Spiel mit der Wahrheit – und Proteste gegen Is...
78,Polizist schlägt Frau bei Demo in Berlin


**Content-Preview**

In [995]:
ts_content_preview_tag = ts_soup.find_all('p', attrs={'data-kilkaya': True})

In [997]:
title_attr_df(ts_content_preview_tag)

,titel
0,Zwei Männer werden in der Sonnenallee aus ein...
1,Das Hamas-Massaker hat die Region grundlegend...
2,Der Fotograf Luigi Toscano zeigt in Berlin Bi...
3,Das Bundesinnenministerium hat einen islamist...
4,Die Gesellschaft für Freiheitsrechte will Geb...
5,Bei Zohran Mamdani und den Democratic Sociali...
6,"Schröder, Merkel, Scholz – und nun Merz: Vier..."
7,Der Nahe Osten hat sich seit dem 7. Oktober 2...
8,Die RSF-Miliz hat die sudanesische Stadt El F...
9,Das wichtigste und älteste deutsche Dokumenta...


**Datum**

In [ ]:
ts_date_tag = ts_soup.find_all('span', attrs={'data-kilkaya': True})

#### Spiegel

In [1008]:
spiegel="https://www.spiegel.de/thema/nahost/p2/"

In [1010]:
spiegel_html = os.path.join("../input/2025-11-07-spiegel.html")

In [1014]:
with open(spiegel_html, "r", encoding= "utf-8") as f:
    spiegel_text = f.read()

In [1016]:
spiegel_soup = BeautifulSoup(spiegel_text, "html.parser")

**Titel**

In [1024]:
# Large

driver.get(spiegel)

time.sleep(3) 


spiegel_html = driver.page_source
spiegel_soup = BeautifulSoup(spiegel_html, 'lxml')

spiegel_title_tag = spiegel_soup.select('span.align-middle')

print(spiegel_title_tag)

[<span class="inline-flex align-middle leading-none mr-4" data-contains-flags="plus-paid">
<span class="inline-block mr-4 text-primary-base dark:text-dm-primary-base" data-flag-name="plus-paid"><svg aria-hidden="true" class="fill-current" height="20" id="spon-spon-paid-flag-l" role="img" viewbox="0 0 20 20" width="20" xmlns="http://www.w3.org/2000/svg">
<g id="l-splus-flag-78f0aea1-16c7-48c9-84c4-c56919900756">
<rect height="20" id="bg-78f0aea1-16c7-48c9-84c4-c56919900756" rx="1.5" width="20"></rect>
<path clip-rule="evenodd" d="M4 10.684V6.295C4 5.575 4.36 5 5.295 5h3.669c.648 0 1.08.432 1.08 1.151v1.943H8.172v-1.44H5.799v2.303h4.244v4.82c0 .791-.431 1.223-1.295 1.223H5.223C4.36 15 4 14.496 4 13.849v-2.302h1.87v1.798h2.375v-2.662H4z" fill="#fff" fill-rule="evenodd" id="shape-78f0aea1-16c7-48c9-84c4-c56919900756"></path>
<path d="M13.825 7.8h1.35v2.025H17.2v1.35h-2.025V13.2h-1.35v-2.025H11.8v-1.35h2.025V7.8z" fill="#fff" id="shape_2-78f0aea1-16c7-48c9-84c4-c56919900756"></path>
</g>
</

In [1026]:
title_attr_df(spiegel_title_tag)

,titel
0,\n\n\n\n\n\n\n\n\n
1,Ziemlich beste Feinde
2,Israels Militär greift erneut Huthi-Miliz im J...
3,Rund 30 Verletzte bei israelischer Razzia in R...
4,Geiselangehörige warnen Netanyahu-Regierung vo...
5,Israelischer Ausschuss billigt umstrittenen Sp...
6,\n\n\n
7,"»Uns zu unterstellen, es ginge gar nicht um ec..."
8,Libanons Regierung will Waffenmonopol – Hisbol...
9,\n\n\n


**Content-Preview**

In [1039]:
spiegel_content_preview_tag = spiegel_soup.find_all('span', attrs={'data-target-teaser-el': True})

In [1034]:
title_attr_df(spiegel_content_preview_tag)

,titel
0,\nVor der Konferenz zur Zweistaatenlösung\n
1,\nSeit Monaten liefern sich Emmanuel Macron un...
2,"\nVon Leo Klimm, Paris\n"
3,\nKrieg in Nahost\n
4,\nImmer wieder attackieren die von Iran unters...
5,\nMilitäreinsatz im Westjordanland\n
6,\nIsraelische Soldaten sind zu einer Razzia in...
7,"\n»Hungrig, durstig, verängstigt, gefoltert«\n"
8,"\nSie bangen um das Überleben ihrer Söhne, Män..."
9,\nNeue Siedlungen\n


**Datum**

In [1041]:
spiegel_date_tag = spiegel_soup.find_all('span', attrs={'data-auxiliary': True})

In [1043]:
title_attr_df(spiegel_date_tag)

,titel
0,"22. September 2025, 19.28 Uhr"
1,"10. September 2025, 18.37 Uhr"
2,"27. August 2025, 12.42 Uhr"
3,"20. August 2025, 23.35 Uhr"
4,"20. August 2025, 14.16 Uhr"
5,"6. August 2025, 20.06 Uhr"
6,"6. August 2025, 08.21 Uhr"
7,"28. Juli 2025, 21.08 Uhr"
8,"27. Juli 2025, 13.50 Uhr"
9,"26. Juli 2025, 19.06 Uhr"


## Transform

## Test

In [1302]:
sz_df

,id,title,preview,date
0,teaser-httpswwwsueddeutschedepolitikbonn-iran-...,Iran will neues Konsulat in Bonn eröffnen,Das Mullah-Regime plant eine Vertretung am Rhe...,Heute 14:51 Uhr
1,teaser-httpswwwsueddeutschedeprojekteartikelpo...,„Ohne Staatsbürgerschaft hätte ich jetzt richt...,Bundeskanzler Merz sieht für Flüchtlinge aus S...,Heute 14:33 Uhr
2,teaser-httpswwwsueddeutschedepolitiksyrien-un-...,UN heben Sanktionen gegen al-Scharaa auf,Der UN-Sicherheitsrat hat die Sanktionen gegen...,Heute 13:36 Uhr
3,teaser-httpswwwsueddeutschedemeinungpalaestine...,NA,Ein 20-Punkte-Plan schafft noch keinen Frieden...,Heute 8:38 Uhr
4,teaser-httpswwwsueddeutschedepolitikunion-krit...,Herbst des Verdrusses,Eigentlich wollte Friedrich Merz mit einem Her...,6.11.2025
5,teaser-httpswwwsueddeutschedepolitiknahost-isr...,Israel greift im Südlibanon an,Israels Armee hat nach eigenen Angaben begonne...,6.11.2025
6,teaser-httpswwwsueddeutschedepolitikwadephul-u...,Wadephul gibt nicht klein bei,Der Außenminister verteidigt sich in der Union...,5.11.2025
7,teaser-httpswwwsueddeutschedemeinungkommentar-...,NA,Das Vertrauen in die Union sinkt immer mehr – ...,5.11.2025
8,teaser-httpswwwsueddeutschedepolitikisrael-fue...,Fünf Millionen Holocaust-Opfer von Yad Vashem ...,Die israelische Gedenkstätte Yad Vashem hat fü...,4.11.2025
9,teaser-httpswwwsueddeutschedemeinungsyrer-deut...,NA,"Fast alle sind sich einig, dass es richtig ist...",4.11.2025


In [88]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time

from selenium.webdriver.support.ui import WebDriverWait

In [95]:
driver = webdriver.Chrome()
driver.get(sz)

In [62]:
load_element = driver.find_element(By.CLASS_NAME, 'css-1kk8krp')

In [99]:
# --- Dein Setup (Beispiel) ---
now_str = datetime.now().strftime("%Y-%m-%d")
name = "sz"

driver = webdriver.Chrome()
driver.get(sz)

#--------------------Laden der Seite--------------------
target_date = datetime.strptime("08.10.2025", "%d.%m.%Y")
print(f"Suche gestartet. Stoppe, wenn Datum älter als {target_date.strftime('%d.%m.%Y')} ist.")

while True:
    try:
        wait = WebDriverWait(driver, 10)
        load_element = wait.until(
            EC.element_to_be_clickable((By.CLASS_NAME, "css-1kk8krp"))
        )
        load_element.click()

        time.sleep(2) 

        all_dates = driver.find_elements(By.CSS_SELECTOR, 'span.css-0')
        if not all_dates:
            print("Keine Datumsangaben mehr gefunden.")
            break

        last_date_text = all_dates[-1].text.strip()

        try:
            current_date = datetime.strptime(last_date_text, "%d.%m.%Y")
            if current_date < target_date:
                print(f"STOPP: {last_date_text} ist älter als das Ziel.")
                break
        except ValueError:
            pass 

    except TimeoutException:
        print("Button nicht mehr gefunden. Ende erreicht.")
        break


##--------------------Verlängerte Seite speichern--------------------

rohes_html = driver.page_source

driver.quit()

file_name = os.path.join(
    STORAGE_DIR,
    f"{now_str}-{name}.html",
)

try:
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(rohes_html)
    print(f"Erfolgreich gespeichert in: {file_name}")

except Exception as e:
    print(f"Fehler beim Speichern der Datei: {e}")

Suche gestartet. Stoppe, wenn Datum älter als 08.10.2025 ist.
STOPP: 30.9.2025 ist älter als das Ziel.
Erfolgreich gespeichert in: ../input/2025-11-14-sz.html


In [80]:
driver = webdriver.Chrome()
driver.get(sz)

button_element = driver.find_element(By.CLASS_NAME, 'css-1kk8krp')

datum_tag2 = article.select('span.css-0')
    
if datum_tag:

    for d in datum_tag:
            
        datum.append(d.get_text(strip=True))

else:
    datum2.append('NA')

for date in datum_tag2:
    date != "08.10.2025"
    button_element.click()

try:
    wait = WebDriverWait(driver, 10)
    load_element = wait.until(
        EC.element_to_be_clickable(load_element)
    )
except TimeoutException:
    print("Fehler: Button 'Mehr Artikel laden' wurde nach 10 Sek. nicht klickbar.")

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=142.0.7444.162)
Stacktrace:
0   chromedriver                        0x000000010ff269d8 chromedriver + 6482392
1   chromedriver                        0x000000010ff1de1a chromedriver + 6446618
2   chromedriver                        0x000000010f962918 chromedriver + 436504
3   chromedriver                        0x000000010f9b692b chromedriver + 780587
4   chromedriver                        0x000000010f9a9cba chromedriver + 728250
5   chromedriver                        0x000000010f9a9638 chromedriver + 726584
6   chromedriver                        0x000000010fa03eb8 chromedriver + 1097400
7   chromedriver                        0x000000010f9a7daf chromedriver + 720303
8   chromedriver                        0x000000010f9a8ab1 chromedriver + 723633
9   chromedriver                        0x000000010fee2a81 chromedriver + 6204033
10  chromedriver                        0x000000010fee6f55 chromedr

In [66]:
load_element.click()

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=142.0.7444.162); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x00000001036549d8 chromedriver + 6482392
1   chromedriver                        0x000000010364be1a chromedriver + 6446618
2   chromedriver                        0x0000000103090afb chromedriver + 436987
3   chromedriver                        0x0000000103097add chromedriver + 465629
4   chromedriver                        0x000000010309a413 chromedriver + 476179
5   chromedriver                        0x000000010309a4e3 chromedriver + 476387
6   chromedriver                        0x00000001030e54fe chromedriver + 783614
7   chromedriver                        0x00000001030d7c2d chromedriver + 728109
8   chromedriver                 